<a href="https://colab.research.google.com/github/capitanalegria/capi_stream/blob/main/sharon_apple_gpt_streamer_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#created during stream w chat GPT3.5 API and GPT4 03/15/2023 w python 3.9

In [ ]:
#@title Install libraries and components and run locally on an anacando or virtual env: { form-width: "250px", display-mode: "form" }
runtime = "local" #@param ["local"]
##@markdown Name and create project folder.
#project_folder="sharon_apple" #@param {type:"string"}

import os
!pip install gdown --upgrade
import sys
%cd 
if runtime == 'hosted':
    from google.colab import drive
    drive.mount('/content/drive/')
    work_folder='/content/drive/MyDrive/{}/'.format(project_folder)
    !pip install --upgrade setuptools wheel
    !sudo apt install espeak
    !sudo apt install python3-pyaudio
    !!pip install gtts
    RESULTS=work_folder+'RESULTS/'
    os.makedirs(work_folder, exist_ok=True)
    os.makedirs(RESULTS, exist_ok=True)

!pip install openai
!pip install SpeechRecognition
!pip install pyttsx3
!pip install ffmpeg-python

In [ ]:
#@title Run if using conda on Linux or Mac, otherwise install PyAudio from wheel matching Python version. (https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio)
!conda install -c anaconda pyaudio

In [ ]:
#@title Enter your OpenAI api key
yatu="" #@param {type:"string"}

In [ ]:
#@title Talk to your Chatgpt co-host thru mic, End convo with the words "PLEASE STOP"
#@markdown Pick personality or assign one thru text, and choose to create 2 images from convo or not.  { form-width: "100px" }
import webbrowser as wb
import openai
import speech_recognition as sr
import pyttsx3
import requests
from io import BytesIO
import wave
import os.path
from pathlib import Path
from IPython.display import Audio
from IPython.display import display
import urllib.request
from tkinter import filedialog
from tkinter import Tk

make_images = True #@param {type:"boolean"}

openai.api_key = yatu
r = sr.Recognizer()
prompt=""
conversation=""
response=""
image_prompt=""
reponse_summary=""
generated_image=""

personalidad_gpt = "sarcastic cohost" #@param ["sarcastic podcast host", "endearing friend", "guiding wizard", "friendly chatbot", "dark goth boy", "epic gamer", "nerdy politician", "evil programmer", "imperial overlord", "egomaniacal artist", "rider or die partner", "helpful robot"] {allow-input: true}

messages=[{"role":"system", "content": "you are a {}".format(personalidad_gpt)}]

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to recognize speech from microphone input
def recognize_speech():
    speech_recognition=sr.Recognizer()
    with sr.Microphone() as source:
        speech=''
        print("Listening...")
        engine.say("Listening...")
        engine.runAndWait()
        audio=speech_recognition.listen(source,phrase_time_limit=5)
    try:
        speech=speech_recognition.recognize_google(audio)
        print(speech)
        return speech
    except sr.UnknownValueError:
        print("Could not understand your speech, type request")
        speech=input()
        return speech
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
        return None

def interact_w_gpt():
    global messages
    #messages=[]

    # Prompt the user to choose a directory to save the files
    engine.say("Where do you want convo text files and images to be saved?...")
    engine.runAndWait()
    root = Tk()
    root.withdraw()
    save_dir = filedialog.askdirectory(title="Choose a directory to save the images and text file")

    while True:
        print("Speak your input:")
        le_prompt = recognize_speech()
        #le_prompt=input('you:') #uncomment if you want to input prompt manually
        if le_prompt is None:
            continue

        messages.append({"role": "user", "content": le_prompt})

        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.5)
        reply = response.choices[0].message.content

        engine.say(reply)
        print(reply)
        engine.runAndWait()
        messages.append({"role": "assistant", "content": reply})

        title = le_prompt[:20]
        convo_text = os.path.join(save_dir, f'convo_{title}.txt')
        if "Thank you for your time" in reply or "please stop" in le_prompt:
            break

        elif make_images == True:
            image_prompt = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                temperature=0.5,
                messages=[{"role": "user", "content": f'summarize {reply} to 10 words'}]
                )
            image_gen = image_prompt.choices[0].message.content
            generated_image = openai.Image.create(
                prompt=image_gen,
                n=2, #change to however many images you want and adjust on variables below
                size="1024x1024"
                )
            image_urla = generated_image.data[0]['url']
            image_urlb = generated_image.data[1]['url']
            #image_urlc = generated_image.data[2]['url']
            #image_urld = generated_image.data[3]['url']

            urllib.request.urlretrieve(image_urla, os.path.join(save_dir, f'a{title}.jpg'))
            urllib.request.urlretrieve(image_urlb, os.path.join(save_dir, f'b{title}.jpg'))
            #urllib.request.urlretrieve(image_urlc, os.path.join(save_dir, f'c{title}.jpg'))
            #urllib.request.urlretrieve(image_urld, os.path.join(save_dir, f'd{title}.jpg'))
            with open(convo_text, "w") as f:
                f.write(str(messages))

        else:
            with open(convo_text, "w") as f:
                f.write(str(messages))

print('Say something...')
interact_w_gpt()

experimental script below, not tested


In [ ]:
#@title Optionally use TTS AI MODEL to interpret voice input instead of google's speech recognition (not tested):
#install TTS
!pip install TTS

#download model
!wget https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/Coqui-TTS/tts_models--en--ljspeech--tacotron2-DDC.tar.gz
!tar -xvf tts_models--en--ljspeech--tacotron2-DDC.tar.gz

In [ ]:
#@title Talk to GPT w TTS and generate 2 images from convo

import openai
import urllib.request
import os
import speech_recognition as sr
from tkinter import filedialog
from tkinter import Tk
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer

# Initialize the Coqui TTS synthesizer
model_path = "tts_models/en/ljspeech/tacotron2-DDC"
config_path = os.path.join(model_path, "config.json")
model_manager = ModelManager()
tts_model, vocoder_model, tts_config, vocoder_config = model_manager.get_model_config(model_path, config_path)
synthesizer = Synthesizer(tts_model, vocoder_model, tts_config, vocoder_config)

# Function to recognize speech from microphone input
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        synthesizer.tts("Listening...")
        audio = recognizer.listen(source)

    try:
        speech = recognizer.recognize_google(audio)
        print(f"You said: {speech}")
        return speech
    except sr.UnknownValueError:
        print("Could not understand your speech")
        return None
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
        return None

def test_audio_capture():
    test_prompt = recognize_speech()
    if test_prompt is not None:
        print("Audio capture successful. Proceeding to run the script...")
        return True
    else:
        print("Audio capture failed. Please check your microphone and try again.")
        return False

personalidad_gpt = "sarcastic cohost" #@param ["sarcastic podcast host", "endearing friend", "guiding wizard", "friendly chatbot", "dark goth boy", "epic gamer", "nerdy politician", "evil programmer", "imperial overlord", "egomaniacal artist", "rider or die partner", "helpful robot"] {allow-input: true}
messages=[{"role":"system", "content": "you are a {}".format(personalidad_gpt)}]

def interact_w_gpt():
    global messages
    #message=[]

    # Prompt the user to choose a directory to save the files
    root = Tk()
    root.withdraw()
    save_dir = filedialog.askdirectory(title="Choose a directory to save the images and text file")

    while True:
        print("Speak your input:")
        le_prompt = recognize_speech()
        if le_prompt is None:
            continue

        messages.append({"role": "user", "content": le_prompt})

        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0.5)
        reply = response.choices[0].message.content

        synthesizer.tts(reply)
        print(reply)
        messages.append({"role": "assistant", "content": reply})

        title = le_prompt[:20]
        convo_text = os.path.join(save_dir, f'convo_{title}.txt')

        image_prompt = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.5,
            messages=[{"role": "user", "content": f'summarize {reply} to 10 words'}]
        )
        image_gen = image_prompt.choices[0].message.content

        generated_image = openai.Image.create(
            prompt=image_gen,
            n=4,
            size="1024x1024"
        )
        image_urla = generated_image.data[0]['url']
        image_urlb = generated_image.data[1]['url']

        urllib.request.urlretrieve(image_urla, os.path.join(save_dir, f'a{title}.jpg'))
        urllib.request.urlretrieve(image_urlb, os.path.join(save_dir, f'b{title}.jpg'))

        with open(convo_text, "w") as f:
            f.write(str(messages))

        if "Thank you for your time" in reply or "please stop" in le_prompt:
            break

print('Say something...')
interact_w_gpt()